# Extracción de imagenes y mascaras de archivos .mat

La información del dataset usado se encontraba en archivos .mat https://figshare.com/articles/brain_tumor_dataset/1512427/1, este archivo contenia un id de paciente, la imágen de resonancia magnetica, una etiqueta de tipo de afección que sufria el paciente, un vector de coordenadas de los puntos delimitados por los radiologos, una imagen de la mascara del tumor detectado. En nuestro caso es de interes las información de la etiqueta meningiomas (1), la imágen de resonancia magnetica y su correspondiente mascara. 

![Paciente](./Figuras/matOrigen.jpeg)

In [9]:
import numpy as np
import scipy
import scipy.io as sio
import matplotlib.pyplot as plt
import glob
import h5py
import os
import shutil
from scipy import misc
import random
from skimage import io
from PIL import Image
import matplotlib.pyplot as plt

## Crear directorios para las imagenes

Comprobar la ruta de trabajo inicial

In [2]:
dact=os.getcwd()
print(dact)
os.makedirs(dact+'/Imagenes')

/home/user


Crear y comprobar cada carpeta

In [3]:
os.makedirs('Imagenes/l1')
print(os.path.isdir('Imagenes/l1'))
os.makedirs('Imagenes/l2')
print(os.path.isdir('Imagenes/l2'))
os.makedirs('Imagenes/l3')
print(os.path.isdir('Imagenes/l3'))

True
True
True


Eliminar carpetas de un directorio

In [4]:
# os.rmdir("Imagenes/l1")
# os.rmdir("Imagenes/l2")
# os.rmdir("Imagenes/l3")

comparar imagenes para verificar que mantenga la misma cantidad de informacion. 

## Nota: 
El parametro I se usa para determinar el nombre de la carpeta que contiene todos los archivos .mat, si se desea procesar todas las imagenes en una sola tanda, es posible que al hacerlo desde una máquina virtual aparezca un error en el kernel de su máquina virtual y no termine el proceso de extracción de la información.

In [ ]:
#Images=glob.glob('Images/*.mat')
I="I1-3064/"
Images=glob.glob("Images/"+I+"*.mat")
print "images",len(Images)
archive=Images[1]
with h5py.File(archive, 'r') as f:
    data=f['cjdata']
    
    lb=int(data['label'][0][0])
    
    im=np.array(data['image'], np.dtype('int16'))
    print(im.min())
    print(im.max())
    max=im.max()
    img=scipy.misc.toimage(im, cmin=0.0, cmax=max, mode='L')
    img.save("im.png")
    print("tipo",type(img))
   
    mas=np.array(data['tumorMask'], np.dtype('float64'))
    min=mas.min()
    max=mas.max()
    mas=scipy.misc.toimage(mas, cmin=0.0, cmax=1)
    io.imsave("mas.png",mas)
    
    print type(im)

## Extrar imagenes, mascaras y nombre de cada archivo .mat

Quedan guardados en un tres listas ima[],mas[],lab[]

In [ ]:
#Images=glob.glob('Images/*.mat')
Images=glob.glob("Images/"+I+"*.mat")

ima=[]
mas=[]
lab=[]
nombre=[]
#print "Tamaño",len(Images)
i=0  
con=3064#control de iteraciones para no procesar todas las imagenes
l1=0 #control de etiquetas 1,2,3
l2=0
l3=0

#Extraer imagen, mascara y etiqueta del .mat
for archive in Images:
    with h5py.File(archive, 'r') as f:
        data=f['cjdata']
        
        inicio=7+10
        mat=4
        fin=len(archive)-inicio-mat
        
        nombre += [archive[inicio:inicio+fin]]
        ima    += [np.array(data['image'], np.dtype('int16'))]    
        mas    += [np.array(data['tumorMask'], np.dtype('float64'))]
        lab    += [int(data['label'][0][0])]
        i=i+1
        if i==con:
            break

## Separar la informacion por etiqueta

In [ ]:
ima1=[]
mas1=[]
lab1=[]
nom1=[]

ima2=[]
mas2=[]
lab2=[]
nom2=[]
    
ima3=[]
mas3=[]
lab3=[]
nom3=[]
            
for i in range(len(ima)):
    
    #Separar informacion por etiquetas
    if lab[i]==1:
        ima1+=[ima[i]]
        mas1+=[mas[i]]
        lab1+=[lab[i]]
        nom1+=[nombre[i]]
    if lab[i]==2:
        ima2+=[ima[i]]
        mas2+=[mas[i]]
        lab2+=[lab[i]]
        nom2+=[nombre[i]]
    if lab[i]==3:
        ima3+=[ima[i]]
        mas3+=[mas[i]]
        lab3+=[lab[i]]
        nom3+=[nombre[i]]

## Separar los datos en train y test
Al final quedan diez y ocho listas correspondientes a train y test para imagen, mascara y label, la seleccion se hace de manera aleatoria para evitar sesgos en la información. 

In [ ]:
test_im1=[]
test_mas1=[]
test_lab1=[]
test_nom1=[]

n=len(ima1)
porcentaje=n-int(round(n*0.7))

for _ in range(0,porcentaje):
    seleccion = random.randint(0,len(ima1)-1)
    
    test_im1   += [ima1[seleccion]]
    ima1.pop(seleccion)
    
    test_mas1  += [mas1[seleccion]]
    mas1.pop(seleccion)
    
    test_lab1 += [lab1[seleccion]]
    lab1.pop(seleccion)
    
    test_nom1 += [nom1[seleccion]]
    nom1.pop(seleccion)
    
test_im2=[]
test_mas2=[]
test_lab2=[]
test_nom2=[]

n=len(ima2)

porcentaje=n-int(round(n*0.7))
for _ in range(0,porcentaje):
    seleccion = random.randint(0,len(ima2)-1)
    test_im2   += [ima2[seleccion]]
    ima2.pop(seleccion)
    
    test_mas2  += [mas2[seleccion]]
    mas2.pop(seleccion)
    
    test_lab2 += [lab2[seleccion]]
    lab2.pop(seleccion)
    
    test_nom2 += [nom2[seleccion]]
    nom2.pop(seleccion)
    


test_im3=[]
test_mas3=[]
test_lab3=[]
test_nom3=[]

n=len(ima3)

porcentaje=n-int(round(n*0.7))
for _ in range(0,porcentaje):
    seleccion = random.randint(0,len(ima3)-1)
    
    test_im3   += [ima3[seleccion]]
    ima3.pop(seleccion)
    
    test_mas3  += [mas3[seleccion]]
    mas3.pop(seleccion)
    
    test_lab3 += [lab3[seleccion]]
    lab3.pop(seleccion)
    
    test_nom3 += [nom3[seleccion]]
    nom3.pop(seleccion)
    



In [ ]:
print "train ima_1:",len(ima1)
print "test  ima_1:",len(test_im1)
print "train mas_1:",len(mas1)
print "test  mas_1:",len(test_mas1)
print "train nom_1:",len(nom1)
print "test  nom_1:",len(test_nom1)

print "train ima_2:",len(ima2)
print "test  ima_2:",len(test_im2)
print "train mas_2:",len(mas2)
print "test  mas_2:",len(test_mas2)

print "train ima_3:",len(ima3)
print "test  ima_3:",len(test_im3)
print "train mas_3:",len(mas3)
print "test  mas_3:",len(test_mas3)

print "Imagenes totales=",len(ima1)+len(test_im1)+len(ima2)+len(test_im2)+len(ima3)+len(test_im3)

## Guardar en carpetas separadas

In [ ]:
#Train 1
j=0
for i in nom1:
    
    #Carpeta Paciente
    #print i
    os.makedirs('Imagenes/l1/train/'+i)
    
    #Carpeta imagen
    os.makedirs('Imagenes/l1/train/'+i+"/imagen")
    dire="Imagenes/l1/train/"+i+"/imagen"
    print "larg",len(ima1)
    print "j",j
    im=np.array(ima1[j], np.dtype('int16'))
    min=im.min()
    max=im.max()
    img=scipy.misc.toimage(im, cmin=min, cmax=max, mode='L')
    img.save(dire+"/"+i+".png")
    
    
    #Carpeta mascara
    os.makedirs('Imagenes/l1/train/'+i+"/mask")
    dire="Imagenes/l1/train/"+i+"/mask"
    mas=np.array(mas1[j], np.dtype('float64'))
    min=mas.min()
    max=mas.max()
    mas=scipy.misc.toimage(mas, cmin=min, cmax=max)
    io.imsave(dire+"/"+i+".png",mas)

    j=j+1

#Test 1
j=0
for i in test_nom1:
    
    #Carpeta Paciente
    #print i
    os.makedirs('Imagenes/l1/test/'+i)
    
    #Carpeta imagen
    os.makedirs('Imagenes/l1/test/'+i+"/imagen")
    dire="Imagenes/l1/test/"+i+"/imagen"
    im=np.array(test_im1[j], np.dtype('int16'))
    min=im.min()
    max=im.max()
    img=scipy.misc.toimage(im, cmin=min, cmax=max, mode='L')
    img.save(dire+"/"+i+".png")
    
    
    #Carpeta mascara
    os.makedirs('Imagenes/l1/test/'+i+"/mask")
    dire="Imagenes/l1/test/"+i+"/mask"
    mas=np.array(test_mas1[j], np.dtype('float64'))
    min=mas.min()
    max=mas.max()
    mas=scipy.misc.toimage(mas, cmin=min, cmax=max)
    io.imsave(dire+"/"+i+".png",mas)

    j=j+1

In [ ]:
#Train 2
j=0
for i in nom2:
    
    #Carpeta Paciente
    #print i
    os.makedirs('Imagenes/l2/train/'+i)
    
    #Carpeta imagen
    os.makedirs('Imagenes/l2/train/'+i+"/imagen")
    dire="Imagenes/l2/train/"+i+"/imagen"
    print "larg",len(ima2)
    print "j",j
    im=np.array(ima2[j], np.dtype('int16'))
    min=im.min()
    max=im.max()
    img=scipy.misc.toimage(im, cmin=min, cmax=max, mode='L')
    img.save(dire+"/"+i+".png")
    
    
    #Carpeta mascara
    os.makedirs('Imagenes/l2/train/'+i+"/mask")
    dire="Imagenes/l2/train/"+i+"/mask"
    mas=np.array(mas2[j], np.dtype('float64'))
    min=mas.min()
    max=mas.max()
    mas=scipy.misc.toimage(mas, cmin=min, cmax=max)
    io.imsave(dire+"/"+i+".png",mas)

    j=j+1

#Test 2
j=0
for i in test_nom2:
    
    #Carpeta Paciente
    #print i
    os.makedirs('Imagenes/l2/test/'+i)
    
    #Carpeta imagen
    os.makedirs('Imagenes/l2/test/'+i+"/imagen")
    dire="Imagenes/l2/test/"+i+"/imagen"
    im=np.array(test_im2[j], np.dtype('int16'))
    min=im.min()
    max=im.max()
    img=scipy.misc.toimage(im, cmin=min, cmax=max, mode='L')
    img.save(dire+"/"+i+".png")
    
    
    #Carpeta mascara
    os.makedirs('Imagenes/l2/test/'+i+"/mask")
    dire="Imagenes/l2/test/"+i+"/mask"
    mas=np.array(test_mas2[j], np.dtype('float64'))
    min=mas.min()
    max=mas.max()
    mas=scipy.misc.toimage(mas, cmin=min, cmax=max)
    io.imsave(dire+"/"+i+".png",mas)

    j=j+1

In [ ]:
#Train 3
j=0
for i in nom3:
    
    #Carpeta Paciente
    #print i
    os.makedirs('Imagenes/l3/train/'+i)
    
    #Carpeta imagen
    os.makedirs('Imagenes/l3/train/'+i+"/imagen")
    dire="Imagenes/l3/train/"+i+"/imagen"
    print "larg",len(ima1)
    print "j",j
    im=np.array(ima3[j], np.dtype('int16'))
    min=im.min()
    max=im.max()
    img=scipy.misc.toimage(im, cmin=min, cmax=max, mode='L')
    img.save(dire+"/"+i+".png")
    
    
    #Carpeta mascara
    os.makedirs('Imagenes/l3/train/'+i+"/mask")
    dire="Imagenes/l3/train/"+i+"/mask"
    mas=np.array(mas3[j], np.dtype('float64'))
    min=mas.min()
    max=mas.max()
    mas=scipy.misc.toimage(mas, cmin=min, cmax=max)
    io.imsave(dire+"/"+i+".png",mas)

    j=j+1

#Test 3
j=0
for i in test_nom3:
    
    #Carpeta Paciente
    #print i
    os.makedirs('Imagenes/l3/test/'+i)
    
    #Carpeta imagen
    os.makedirs('Imagenes/l3/test/'+i+"/imagen")
    dire="Imagenes/l3/test/"+i+"/imagen"
    im=np.array(test_im3[j], np.dtype('int16'))
    min=im.min()
    max=im.max()
    img=scipy.misc.toimage(im, cmin=min, cmax=max, mode='L')
    img.save(dire+"/"+i+".png")
    
    
    #Carpeta mascara
    os.makedirs('Imagenes/l3/test/'+i+"/mask")
    dire="Imagenes/l3/test/"+i+"/mask"
    mas=np.array(test_mas3[j], np.dtype('float64'))
    min=mas.min()
    max=mas.max()
    mas=scipy.misc.toimage(mas, cmin=min, cmax=max)
    io.imsave(dire+"/"+i+".png",mas)

    j=j+1